In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re

url = f'https://fsca.swissmedic.ch/mep/#/?q=&from=1934-04-18&to=2023-04-19&onlyUpdates=false&sort=PUBLICATION_DATE&direction=DESC'
page = requests.get(url)

In [15]:
def scroll_down(driver_main, test=0):
    """A method for scrolling the page."""

    # Get scroll height.
    last_height = driver_main.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom.
        driver_main.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver_main.execute_script("return document.body.scrollHeight")

        if test == 1:
            break

        if new_height == last_height:
            break

        last_height = new_height

In [16]:
driver_main = webdriver.Chrome(ChromeDriverManager().install())
driver_main.get(url)
print("Start: Scroll Down")
scroll_down(driver_main, 1)
print("Complete: Scroll Down")
time.sleep(3)  # quick way to make sure website has been loaded completely, without scroll_down

# make sure the whole page has been loaded by scrolling down
print("Start: Encoding Content")
content_main = driver_main.page_source.encode('utf-8').strip()
print("Complete: Encoding Content")
soup_main = BeautifulSoup(content_main,"html.parser")

driver_main.quit()

/var/folders/2g/mmrh97td0m70h3qg8l_5gcbr0000gn/T/ipykernel_1475/277743169.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_main = webdriver.Chrome(ChromeDriverManager().install())


Start: Scroll Down
Complete: Scroll Down
Start: Encoding Content
Complete: Encoding Content


In [17]:
soup_main

<html><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title>Liste Rückrufe und andere Sicherheitsmassnahmen (FSCA)</title><base href="./"/><link href="favicon.ico" rel="icon" type="image/x-icon"/><link href="styles.26a09f56d3d4787b1246.bundle.css" rel="stylesheet"/><style>.cmp-mainnavigation[_ngcontent-c0]{margin-bottom:15px}</style><style>header[_ngcontent-c1]{padding:10px 15px}header[_ngcontent-c1]   .navbar[_ngcontent-c1]{padding:0}header[_ngcontent-c1]   .navbar[_ngcontent-c1]   .navbar-toggler[_ngcontent-c1]{right:.1rem;margin-top:1.5em}header[_ngcontent-c1]   .navbar[_ngcontent-c1]   .navbar-brand[_ngcontent-c1]   a[_ngcontent-c1]{float:left;width:200px}header[_ngcontent-c1]   .navbar[_ngcontent-c1]   .navbar-brand[_ngcontent-c1]   a[_ngcontent-c1]   .cmp-brand-logo[_ngcontent-c1]{float:left;height:auto;margin:0;padding:0}header[_ngcontent-c1]   .navbar[_ngcontent-c1]   .cmp-se

In [18]:
# class="table search-result table-responsive table-hover table-striped"
soup_edit = soup_main.find('table', attrs={'class':'table search-result table-responsive table-hover table-striped'})

soup_edit

<table _ngcontent-c6="" class="table search-result table-responsive table-hover table-striped" id="resultTable">
<thead _ngcontent-c6="">
<tr _ngcontent-c6="">
<th _ngcontent-c6="" class="row-date" id="APP.COMMON.DATE">Datum</th>
<th _ngcontent-c6="" class="row-manufacturer" id="PUBLICATION-SEARCH-RESULT.MANUFACTURER">Hersteller</th>
<th _ngcontent-c6="" class="row-trade-name" id="PUBLICATION-SEARCH-RESULT.TRADE_NAME">Handelsname</th>
<th _ngcontent-c6="" class="row-description" id="PUBLICATION-SEARCH-RESULT.PRODUCT_DESCRIPTION">Beschreibung</th>
<th _ngcontent-c6="" class="row-information" id="PUBLICATION-SEARCH-RESULT.INFORMATION">Informationen</th>
<th _ngcontent-c6="" class="row-refId" id="PUBLICATION-SEARCH-RESULT.SWISSMEDIC_REF">Swissmedic Referenz</th>
</tr>
</thead>
<tbody _ngcontent-c6="">
<!-- --><tr _ngcontent-c6="">
<td _ngcontent-c6="" data-title="Datum">
          19.04.2023
          <!-- -->
</td>
<td _ngcontent-c6="" data-title="Hersteller">Philips Healthcare (Suzhou) 

In [19]:
header_list = []
for header in soup_edit.find('thead').find_all('th'):
    header_list.append(header.text)
df = pd.DataFrame(columns=header_list)
df

,Datum,Hersteller,Handelsname,Beschreibung,Informationen,Swissmedic Referenz


In [20]:
# input: string
# output: string
# Removes '\n' and multiple spaces.
def clean_string(string):
    string = re.sub(' +', ' ', string.replace('\n', ''))
    return string

for entry in soup_edit.find('tbody').find_all('tr'):
    date = clean_string(entry.find_all('td')[0].text)
    producer = clean_string(entry.find_all('td')[1].text)
    trade_name = clean_string(entry.find_all('td')[2].text)
    description = clean_string(entry.find_all('td')[3].text)
    info = clean_string(entry.find_all('td')[4].text)
    swissmedic_ref = clean_string(entry.find_all('td')[5].text)
    df.loc[len(df)] = [date, producer, trade_name, description, info, swissmedic_ref]

df

,Datum,Hersteller,Handelsname,Beschreibung,Informationen,Swissmedic Referenz
0,19.04.2023,"Philips Healthcare (Suzhou) Co., Ltd.","Incisive CT, CT3500 Model: 728143, 728144, 72...","MD: Full-body MRI system, superconducting magnet",EN-1,Vk_20230413_19
1,19.04.2023,Fresenius Kabi AG,Injectomatspritze mit Kanüle Model: M93000000...,MD: SyringeMD: SyringeMD: Syringe,DE-1,Vk_20230412_31
2,19.04.2023,Fehling Instruments GmbH & Co. KG,"Osteobioptome Model: NKS-6 Lot: 13J13544, 14C...","MD: Forceps, Biopsy",DE-1,Vk_20230417_02
3,14.04.2023 (Updated 19.04.2023),Iradimed Corporation,1057 MRiduim MRI Infusion Syringe Adapter Set...,"MD: Electric infusion pump administration set,...",DE-1FR-1IT-1EN-1,Vk_20230406_20
4,19.04.2023,"Beckman Coulter Ireland, Inc.","Bicarbonate Model: OSR6190, OSR629 Lot: all",IVD: Other Clinical Chemistry,FR-1,Vk_20230414_03
...,...,...,...,...,...,...
95,08.03.2023,Hamilton Bonaduz AG,Hamilton-C6 Model: 160021,MD: Respiratory And Anaesthesia Devices - Other,DE-1FR-1IT-1EN-1,Vk_20230208_14
96,08.03.2023,Radiometer Medical,AQURE system Model: 933-599 SW-Version: 2.5.2...,IVD: Medical equipment clinical data interfaci...,DE-1FR-1IT-1EN-1,Vk_20230301_23
97,14.02.2023 (Updated 07.03.2023),Caesarea Medical Electronics Limited,BD BodyGuard MicroSets Model: See FSN Lot: all,"MD: Electric infusion pump administration set,...",DE-1DE-2FR-1FR-2IT-1IT-2,Vk_20230207_14
98,07.03.2023,MAQUET Cardiopulmonary GmbH,Heater-Cooler Unit HCU 40 High Voltage Model:...,MD: Cardiopulmonary bypass system heat exchang...,DE-1EN-1,Vk_20230306_19


In [9]:
len(df)

100

In [22]:
df.to_csv("swissmedic_sample.csv", index=False)